<a href="https://colab.research.google.com/github/luanfelixcoding/climate-change-impact-economy/blob/master/climate_change_impact_economy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Link para base de dados:** https://www.kaggle.com/datasets/tarunrm09/climate-change-indicators

**Link para base de dados:** https://www.kaggle.com/datasets/sazidthe1/global-inflation-data?resource=download

**Link para base de dados:** https://www.kaggle.com/datasets/samithsachidanandan/average-monthly-surface-temperature-1940-2024

In [ ]:
"""# conectando com o drive
from google.colab import drive
drive.mount('/content/drive')"""

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# conectando com o drive
from google.colab import drive
drive.mount('/content/drive')

# Criar as pastas
!mkdir -p /content/drive/MyDrive/datasets/climate_change_dataset
!mkdir -p /content/drive/MyDrive/datasets/global_inflation_dataset

# Instalar o Kaggle CLI (caso não esteja instalado)
!pip install kaggle

# Baixar os datasets do Kaggle
!kaggle datasets download -d tarunrm09/climate-change-indicators -p /content/drive/MyDrive/datasets/climate_change_dataset --unzip
!kaggle datasets download -d sazidthe1/global-inflation-data -p /content/drive/MyDrive/datasets/global_inflation_dataset --unzip

# Renomear os arquivos baixados
!mv /content/drive/MyDrive/datasets/climate_change_dataset/*.csv /content/drive/MyDrive/datasets/climate_change_dataset/climate_change_indicators.csv
!mv /content/drive/MyDrive/datasets/global_inflation_dataset/*.csv /content/drive/MyDrive/datasets/global_inflation_dataset/global_inflation_data.csv

!echo "Pastas e arquivos configurados com sucesso!"


In [1]:
# importando bibliotecas
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from sklearn.linear_model import LinearRegression

In [26]:
climate_change_file_path: str = "/content/drive/My Drive/datasets/climate_change_dataset/climate_change_indicators.csv"
global_inflation_file_path : str = "/content/drive/My Drive/datasets/global_inflation_dataset/global_inflation_data.csv"

# datasets
df_climate = pd.read_csv(climate_change_file_path)
df_economy = pd.read_csv(global_inflation_file_path)

df_climate.info(), df_economy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 225 entries, 0 to 224
Data columns (total 72 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ObjectId             225 non-null    int64  
 1   Country              225 non-null    object 
 2   ISO2                 223 non-null    object 
 3   ISO3                 225 non-null    object 
 4   Indicator            225 non-null    object 
 5   Unit                 225 non-null    object 
 6   Source               225 non-null    object 
 7   CTS_Code             225 non-null    object 
 8   CTS_Name             225 non-null    object 
 9   CTS_Full_Descriptor  225 non-null    object 
 10  F1961                188 non-null    float64
 11  F1962                189 non-null    float64
 12  F1963                188 non-null    float64
 13  F1964                188 non-null    float64
 14  F1965                188 non-null    float64
 15  F1966                192 non-null    flo

(None, None)

In [5]:
# verificando valores nulos em cada coluna
def checking_null_values(df: pd.DataFrame) -> None:
  for col in df.columns:
    if df[col].isnull().sum() > 0:
      print(f"{df[col].name} = {df[col].isnull().sum()}")

In [36]:
# Pegando apenas as colunas que representam anos
years = [col for col in df_climate.columns if col.startswith('F')]

# Derrete o dataset em variaveis e valores
# 'Country' e 'ISO3' mantemos, nomeamos uma coluna como 'Year' que recebe a variavel 'years'
df_climate_melted = df_climate.melt(
    id_vars=['Country', 'ISO3'],
    value_vars=years,
    var_name='Year',
    value_name='Temperature_Change'
)
# Na coluna 'Year' extraindo todos os numeros e convertendo para int, "(\d+)" permite extrair numeros compostos
df_climate_melted['Year'] = df_climate_melted['Year'].str.extract("(\d+)").astype(int)
df_climate_melted.rename(columns={'Country': 'country_name'}, inplace=True)
# checking_null_values(df_climate_melted) # output: Temperature_Change = 1490

# Removendo valores nulos; inplace=True faz as alterações no proprio df, inplace=False cria outro df com os valores removidos
df_climate_melted.dropna(subset=['Temperature_Change'], inplace=True)
checking_null_values(df_climate_melted) # output: Temperature_Change = 0
df_climate_melted.head()

,country_name,ISO3,Year,Temperature_Change
0,"Afghanistan, Islamic Rep. of",AFG,1961,-0.113
1,Albania,ALB,1961,0.627
2,Algeria,DZA,1961,0.164
3,American Samoa,ASM,1961,0.079
4,"Andorra, Principality of",AND,1961,0.736


## Plotando Gráfico referente as mudanças climáticas durante os anos

In [37]:
# Agrupando a coluna 'Year' e pegando a media de seus valores em conjunto
global_temp_trend = df_climate_melted.groupby('Year', as_index=False)['Temperature_Change'].mean().sort_values(by='Year')

# Criando a visualização
fig_global = go.Figure()

fig_global.add_trace(go.Scatter(
    x=global_temp_trend['Year'],
    y=global_temp_trend['Temperature_Change'],
    mode='lines+markers',
    line=dict(color='red', width=2),  # Linha vermelha
    marker=dict(color='black', size=5),  # Pontos pretos
    name='Tendência Global'
))


fig_global.update_layout(
    title="Tendência Global da Mudança de Temperatura (1961-2022)",
    xaxis_title="Ano",
    yaxis_title="Mudança Média de Temperatura (°C)",
    template="plotly_white",
    showlegend=True
)

# Exibir o gráfico
fig_global.show()

In [40]:
# Calculando a variação de temperatura por país (máximo - mínimo)
df_climate_variation = df_climate_melted.groupby("country_name")["Temperature_Change"].agg(lambda x: x.max() - x.min())

# Selecionando os 10 países mais afetados (com maior variação)
top_10_countries = df_climate_variation.nlargest(10).index
df_top_10_countries = df_climate_melted[df_climate_melted['country_name'].isin(top_10_countries)].sort_values(by=['Year'])

# Criando o gráfico interativo
fig_countries = px.line(df_top_10_countries, x="Year", y="Temperature_Change", color="country_name",
                        title="Comparação dos Países com Maior Variação ao longo dos Anos",
                        labels={"Temperature_Change": "Mudança Média de Temperatura (°C)", "Year": "Ano"},
                        markers=True)

fig_countries.update_traces(line=dict(width=2))
fig_countries.show()


# Dados sobre o dataset de Economia

In [42]:
df_economy_melted = df_economy.melt(
    id_vars=['country_name'],
    value_vars=[str(col) for col in range(1980, 2025)],
    var_name='Year',
    value_name='Inflation'
)
df_economy_melted['Year'] = df_economy_melted['Year'].astype(int)
df_economy_melted.head()

,country_name,Year,Inflation
0,Afghanistan,1980,13.4
1,Albania,1980,NaN
2,Algeria,1980,9.7
3,Andorra,1980,NaN
4,Angola,1980,46.7


In [71]:
# Combine and clean data
df_combined = pd.merge(df_climate_melted, df_economy_melted, on=['country_name', 'Year'], how='inner').dropna()

# Calculate correlation
correlation = df_combined[['Temperature_Change', 'Inflation']].corr().iloc[0, 1]

# Create scatter plot with trendline
fig = px.scatter(
    df_combined,
    x='Temperature_Change',
    y='Inflation',
    opacity=0.5,
    title=f'Correlação entre Mudança de Temperatura e Inflação: {correlation:.2f}',
    labels={'Temperature_Change': 'Mudança Média de Temperatura (°C)', 'Inflation': 'Inflação (%)'},
    template='plotly_white',
    trendline='ols',
    width=800,
    height=500
)

# Add grid and show plot
fig.update_xaxes(showgrid=True, gridcolor='lightgray')
fig.update_yaxes(showgrid=True, gridcolor='lightgray')
fig.show()

In [74]:
# Análise de Volatilidade Econômica
top_volatility = df_economy_melted.groupby('country_name')['Inflation'].std().nlargest(10).reset_index()

# Gráfico de barras com Plotly
fig = px.bar(
    top_volatility,
    y='country_name',
    x='Inflation',
    orientation='h',
    color='Inflation',
    color_continuous_scale='Viridis',
    title="Top 10 Países com Maior Volatilidade Econômica (Inflação)",
    labels={'Inflation': 'Desvio Padrão da Inflação (%)', 'country_name': 'País'},
    width=800,
    height=500
)

# Ajustes visuais
fig.update_xaxes(showgrid=True, gridcolor='lightgray', zeroline=True, zerolinecolor='gray')
fig.show()

In [67]:
df_combined = pd.merge(df_climate_melted, df_economy_melted, on=['country_name', 'Year'], how='inner')
df_combined_clean = df_combined.dropna()
df_combined_clean.head()

,country_name,ISO3,Year,Temperature_Change,Inflation
1,Algeria,DZA,1980,0.232,9.7
2,Angola,AGO,1980,0.279,46.7
3,Antigua and Barbuda,ATG,1980,0.646,19.0
5,Australia,AUS,1980,0.887,10.1
6,Austria,AUT,1980,-0.274,6.3


In [75]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from sklearn.linear_model import LinearRegression
import numpy as np

# Caminhos dos arquivos
df_climate = pd.read_csv("/content/drive/My Drive/datasets/climate_change_dataset/climate_change_indicators.csv")
df_economy = pd.read_csv("/content/drive/My Drive/datasets/global_inflation_dataset/global_inflation_data.csv")

# Limpeza e transformação dos dados climáticos
df_climate_filtered = df_climate[['Country', 'ISO3'] + [col for col in df_climate.columns if col.startswith('F')]]
df_climate_filtered.rename(columns={'Country': 'country_name'}, inplace=True)

df_climate_melted = df_climate_filtered.melt(
    id_vars=['country_name', 'ISO3'],
    var_name='Year',
    value_name='Temperature_Change'
)
df_climate_melted['Year'] = df_climate_melted['Year'].str.extract("(\d+)").astype(int)
df_climate_melted.dropna(subset=['Temperature_Change'], inplace=True)

# Limpeza e transformação dos dados econômicos
df_economy_filtered = df_economy.drop(columns=['indicator_name'])
df_economy_melted = df_economy_filtered.melt(
    id_vars=['country_name'],
    var_name='Year',
    value_name='Inflation'
)
df_economy_melted['Year'] = df_economy_melted['Year'].astype(int)
df_economy_melted.dropna(subset=['Inflation'], inplace=True)

# Análise da tendência global da temperatura
global_temp_trend = df_climate_melted.groupby('Year', as_index=False)['Temperature_Change'].mean()
fig_global = go.Figure()
fig_global.add_trace(go.Scatter(
    x=global_temp_trend['Year'],
    y=global_temp_trend['Temperature_Change'],
    mode='lines+markers',
    line=dict(color='red', width=2),
    marker=dict(color='black', size=5),
    name='Tendência Global'
))
fig_global.update_layout(
    title="Tendência Global da Mudança de Temperatura (1961-2022)",
    xaxis_title="Ano",
    yaxis_title="Mudança Média de Temperatura (°C)",
    template="plotly_white",
    showlegend=True
)
fig_global.show()

# Análise da correlação entre temperatura e inflação
df_combined = pd.merge(df_climate_melted, df_economy_melted, on=['country_name', 'Year'], how='inner')
df_combined.dropna(inplace=True)
correlation = df_combined[['Temperature_Change', 'Inflation']].corr().iloc[0, 1]
fig_correlation = px.scatter(
    df_combined,
    x='Temperature_Change',
    y='Inflation',
    opacity=0.5,
    title=f'Correlação entre Mudança de Temperatura e Inflação: {correlation:.2f}',
    labels={'Temperature_Change': 'Mudança Média de Temperatura (°C)', 'Inflation': 'Inflação (%)'},
    template='plotly_white',
    trendline='ols'
)
fig_correlation.show()

# Análise dos 10 países com maior variação de temperatura
df_climate_variation = df_climate_melted.groupby("country_name")["Temperature_Change"].agg(lambda x: x.max() - x.min())
top_10_countries = df_climate_variation.nlargest(10).index
df_top_10_countries = df_climate_melted[df_climate_melted['country_name'].isin(top_10_countries)].sort_values(by=['Year'])
fig_countries = px.line(
    df_top_10_countries, x="Year", y="Temperature_Change", color="country_name",
    title="Comparação dos Países com Maior Variação ao longo dos Anos",
    labels={"Temperature_Change": "Mudança Média de Temperatura (°C)", "Year": "Ano"},
    markers=True
)
fig_countries.show()

# Análise de volatilidade econômica
top_volatility = df_economy_melted.groupby('country_name')['Inflation'].std().nlargest(10).reset_index()
fig_volatility = px.bar(
    top_volatility,
    y='country_name',
    x='Inflation',
    orientation='h',
    color='Inflation',
    color_continuous_scale='Viridis',
    title="Top 10 Países com Maior Volatilidade Econômica (Inflação)",
    labels={'Inflation': 'Desvio Padrão da Inflação (%)', 'country_name': 'País'}
)
fig_volatility.show()

# Previsão do impacto futuro da temperatura na inflação
X = df_combined[['Temperature_Change']]
y = df_combined['Inflation']
model = LinearRegression()
model.fit(X, y)
df_combined['Predicted_Inflation'] = model.predict(X)
fig_prediction = px.scatter(
    df_combined,
    x='Temperature_Change',
    y='Predicted_Inflation',
    opacity=0.5,
    title="Predição de Inflação Baseada no Aumento de Temperatura",
    labels={'Temperature_Change': 'Mudança Média de Temperatura (°C)', 'Predicted_Inflation': 'Inflação Prevista (%)'},
    template='plotly_white'
)
fig_prediction.show()


<ipython-input-75-119696c2b345>:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [86]:
# Preparação dos dados para ML
X = global_temp_trend[['Year']]
y = global_temp_trend['Temperature_Change']

# Criando o modelo de regressão linear
model = LinearRegression()
model.fit(X, y)

# Fazendo previsões para 2024 e 2025
future_years = np.array([[2023], [2024], [2025]])
predictions = model.predict(future_years)

# Criando o gráfico interativo com Plotly
fig = go.Figure()

# Adicionando os dados reais
fig.add_trace(go.Scatter(
    x=X['Year'],
    y=y,
    mode='markers+lines',
    marker=dict(size=6, color='blue'),
    line=dict(color='blue', width=2),
    name="Temperatura Média Observada"
))

# Adicionando a tendência linear
fig.add_trace(go.Scatter(
    x=X['Year'],
    y=model.predict(X),
    mode='lines',
    line=dict(color='red', dash='dash', width=3),
    name="Tendência Linear"
))

# Adicionando as previsões futuras
fig.add_trace(go.Scatter(
    x=[2023, 2024, 2025],
    y=predictions,
    mode='markers',
    marker=dict(size=10, color='green', symbol='x'),
    name="Previsões para 2023, 2024 e 2025"
))

# Configuração do layout
fig.update_layout(
    title="📈 Predição da Tendência Global de Temperatura",
    xaxis_title="Ano",
    yaxis_title="Mudança Média de Temperatura (°C)",
    template="plotly_white",
    showlegend=True,
    hovermode="x"
)

# Exibir gráfico interativo
fig.show()

# Exibir previsões numéricas
print(f"Previsão para 2023: {predictions[0]:.2f} °C")
print(f"Previsão para 2024: {predictions[1]:.2f} °C")
print(f"Previsão para 2025: {predictions[2]:.2f} °C")

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning:

X does not have valid feature names, but LinearRegression was fitted with feature names



Previsão para 2023: 1.34 °C
Previsão para 2024: 1.36 °C
Previsão para 2025: 1.39 °C


### TODO: baixar dataset - https://www.kaggle.com/datasets/samithsachidanandan/average-monthly-surface-temperature-1940-2024
DEPOIS: analisar o dataset e fazer predição para o ano de 2025